In [ ]:
""" 
1- Preciso excluir as linhas de 'nível' 'SECUNDARIA'. Acabei mandando a base toda pra rodar. 
"""

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

df_main = pd.read_csv('./csv/ESTANDAR_BASICA_I2324.csv')
df_scraping = pd.read_csv('./csv/base_scraping.csv')
extraction = pd.read_csv('./csv/extraction.csv')
df_high_mx = pd.read_csv('./csv/high_touch_mx.csv')
ladas_mexico = pd.read_csv('./csv/ladas_mexico.csv')

In [ ]:
display(df_high_mx.head(1))

In [ ]:
# Faz o merge adicionando a coluna 'telefone_scraping' à base de 12k

# Eliminar duplicatas de clave cct
df_scraping_unique = df_scraping.drop_duplicates(subset='clavecct', keep='first')

df_high_mx = extraction.merge(
    df_scraping_unique[['clavecct', 'telefone_scraping']],
    on='clavecct',
    how='left'
)

In [ ]:
# Verificar quantos números foram adicionados através da contagem de ocorrências válidas para a coluna 'telefone_scraping'.

tel_adicionados = df_high_mx[df_high_mx['telefone_scraping'].notna()]   
porcentagem = (len(tel_adicionados) / len(df_high_mx)) * 100 
print(f'O dataframe tem {len(df_high_mx)} linhas.')                    
print(f'Foram adicionados {len(tel_adicionados)} números de telefone novos ({porcentagem:.1f}%).')

In [ ]:
import phonenumbers

# Cria uma cópia para evitar warning
tel_validos = df_high_mx[df_high_mx['telefone_scraping'].notna()].copy()

def validar_telefone(numero, codigo_pais='MX'):
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        return phonenumbers.is_valid_number(parsed)
    except phonenumbers.NumberParseException:
        return False

# Aplica a validação de forma segura
tel_validos['telefone_valido'] = tel_validos['telefone_scraping'].apply(
    lambda x: validar_telefone(x, 'MX')
)

# Exibe os resultados
print(f'O dataframe tem {len(df_high_mx)} escolas.') 

print(f'Total de telefones adicionados: {len(tel_validos)}')
print("Total de telefones válidos:", tel_validos['telefone_valido'].sum())

proporcao_total_real = (tel_validos['telefone_valido'].sum() / len(df_high_mx)) * 100
print(f'Proporção de telefones válidos adicionados: {proporcao_total_real:.1f}%')

proporcao = tel_validos['telefone_valido'].mean() * 100
print(f"Proporção de válidos em relação a inválidos: {proporcao:.1f}%")

telefones_invalidos = tel_validos[~tel_validos['telefone_valido']]
print(f'Quantidade de telefones inválidos: {len(telefones_invalidos)}')

In [ ]:
print('Telefones inválidos abaixo:')
display(telefones_invalidos['telefone_scraping'])

In [ ]:
# Filtra os inválidos
telefones_invalidos = tel_validos[~tel_validos['telefone_valido']].copy()

# Cria coluna com o comprimento dos números
telefones_invalidos['qtde_caracteres'] = telefones_invalidos['telefone_scraping'].str.len()

# Agrupa e conta
resumo = telefones_invalidos.groupby('qtde_caracteres').size().reset_index(name='quantidade')

display(resumo)


In [ ]:
display(df_high_mx.head())

In [ ]:
print(f'O dataframe ladas_mexico tem {len(ladas_mexico)} linhas.')
print(f'Temos {len(ladas_mexico['lada'].unique())} ladas únicas.')
print(f'Temos {len(ladas_mexico['ciudad'].unique())} cidades únicas.')
print(f'Temos {len(ladas_mexico['estado'].unique())} estados únicos.')

In [ ]:
# Esta célula identifica todas as combinações de cidade + estado que possuem mais de uma clave LADA diferente.
# Primeiro, agrupa os dados por cidade e estado e conta quantas LADAs únicas existem em cada grupo.
# Em seguida, filtra apenas os pares com mais de uma LADA.
# Depois, faz um merge com outro agrupamento que coleta todas as LADAs únicas por par cidade + estado (em forma de lista),
# resultando em um DataFrame com as colunas: cidade, estado e a lista de LADAs correspondentes.
# Por fim, exibe quantas dessas combinações existem e mostra o DataFrame resultante.

claves_por_par = (
    ladas_mexico
    .groupby(['ciudad', 'estado'])['lada']
    .nunique()
    .reset_index()
    .query('lada > 1')
    .merge(
        ladas_mexico.groupby(['ciudad', 'estado'])['lada'].unique().reset_index().rename(columns={'lada': 'claves_lada'}),
        on=['ciudad', 'estado']
    )[['ciudad', 'estado', 'claves_lada']]
)

print(f'Temos {len(claves_por_par)} combinações de cidade + estado com claves lada diferentes.')
display(claves_por_par)
display(ladas_mexico.head())

In [ ]:

# teste e valida se é um numero de valido
import pandas as pd
import numpy as np
import phonenumbers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

df_high_mx = pd.read_csv('./csv/high_touch_mx.csv')
ladas_mexico = pd.read_csv('./csv/ladas_mexico.csv')

# Esta célula identifica todas as combinações de cidade + estado que possuem mais de uma clave LADA diferente.
# Primeiro, agrupa os dados por cidade e estado e conta quantas LADAs únicas existem em cada grupo.
# Em seguida, filtra apenas os pares com mais de uma LADA.
# Depois, faz um merge com outro agrupamento que coleta todas as LADAs únicas por par cidade + estado (em forma de lista),
# resultando em um DataFrame com as colunas: cidade, estado e a lista de LADAs correspondentes.
# Por fim, exibe quantas dessas combinações existem e mostra o DataFrame resultante.

claves_por_par = (
    ladas_mexico
    .groupby(['ciudad', 'estado'])['lada']
    .nunique()
    .reset_index()
    .query('lada > 1')
    .merge(
        ladas_mexico.groupby(['ciudad', 'estado'])['lada'].unique().reset_index().rename(columns={'lada': 'claves_lada'}),
        on=['ciudad', 'estado']
    )[['ciudad', 'estado', 'claves_lada']]
)

#print(f'Temos {len(claves_por_par)} combinações de cidade + estado com claves lada diferentes.')
#display(claves_por_par)
#display(ladas_mexico.head())

# Função de validação
def validar_telefone(numero: str, codigo_pais='MX'):
    try:
        parsed = phonenumbers.parse(numero, codigo_pais)
        if phonenumbers.is_valid_number(parsed):
            return numero
    except phonenumbers.NumberParseException:
        pass
    return None

# Normalização prévia dos DataFrames auxiliares
claves_por_par["estado"] = claves_por_par["estado"].astype(str).str.strip().str.lower()
claves_por_par["ciudad"] = claves_por_par["ciudad"].astype(str).str.strip().str.lower()

ladas_mexico["estado"] = ladas_mexico["estado"].astype(str).str.strip().str.lower()
ladas_mexico["ciudad"] = ladas_mexico["ciudad"].astype(str).str.strip().str.lower()

# Função aplicada por linha
def processar_linha(row, claves_por_par, ladas_mexico):
    telefone = row.get("telefone_scraping")
    estado = str(row.get("n_entidad", "")).strip().lower()
    cidade = str(row.get("city", "")).strip().lower()
    resultados = {}

    if isinstance(telefone, str) and len(telefone) in (7, 8):
        # Primeiro: tenta encontrar na claves_por_par
        mask_cp = (claves_por_par["estado"] == estado) & (claves_por_par["ciudad"] == cidade)
        rec_cp = claves_por_par[mask_cp]

        if not rec_cp.empty:
            ddds = rec_cp.iloc[0]["claves_lada"]
            for i, ddd in enumerate(ddds):
                numero = str(ddd) + telefone
                validado = validar_telefone(numero)
                if validado:
                    col = "tel_ddd_adicionado" if i == 0 else f"tel_ddd_adicionado_{i+1}"
                    resultados[col] = validado
            return pd.Series(resultados)

        # Caso contrário: tenta encontrar na ladas_mexico
        mask_lm = (ladas_mexico["estado"] == estado) & (ladas_mexico["ciudad"] == cidade)
        rec_lm = ladas_mexico[mask_lm]

        if not rec_lm.empty:
            ddd = rec_lm.iloc[0]["lada"]
            numero = str(ddd) + telefone
            validado = validar_telefone(numero)
            if validado:
                resultados["tel_ddd_adicionado"] = validado

    return pd.Series(resultados)

# Função principal
def adicionar_ddd(df_high_mx, claves_por_par, ladas_mexico):
    novos_ddds = df_high_mx.apply(
        processar_linha,
        axis=1,
        args=(claves_por_par, ladas_mexico)
    )
    df_ddd_adicionado = pd.concat([df_high_mx, novos_ddds], axis=1)
    return df_ddd_adicionado

# Execução
df_ddd_adicionado = adicionar_ddd(df_high_mx, claves_por_par, ladas_mexico)

# Filtra as colunas que contém 'tel_ddd_adicionado' no nome
colunas_ddd = df_ddd_adicionado.filter(like='tel_ddd_adicionado')

# Quantidade de colunas com 'tel_ddd_adicionado'
num_colunas = len(colunas_ddd.columns)

# Soma total de valores válidos (não nulos) em todas essas colunas
total_validos = colunas_ddd.notna().sum().sum()

print(f"Colunas 'tel_ddd_adicionado' criadas: {num_colunas}")
print(f"Total de valores válidos somados em todas as colunas: {total_validos}")

# Filtra linhas onde a primeira coluna 'tel_ddd_adicionado' tem valor válido (não nulo)
primeira_coluna = colunas_ddd.columns[0]
filtro_primeira_coluna = df_ddd_adicionado[primeira_coluna].notna()

# Retorna o dataframe completo filtrado por esse critério
df_filtrado = df_ddd_adicionado[filtro_primeira_coluna]

display(df_filtrado.head())


In [ ]:
# Filtra as colunas que contém 'tel_ddd_adicionado' no nome
colunas_ddd = df_ddd_adicionado.filter(like='tel_ddd_adicionado')

# Quantidade de colunas com 'tel_ddd_adicionado'
num_colunas = len(colunas_ddd.columns)

# Soma total de valores válidos (não nulos) em todas essas colunas
total_validos = colunas_ddd.notna().sum().sum()

print(f"Colunas 'tel_ddd_adicionado' criadas: {num_colunas}")
print(f"Total de valores válidos somados em todas as colunas: {total_validos}")

# Filtra linhas onde a primeira coluna 'tel_ddd_adicionado' tem valor válido (não nulo)
primeira_coluna = colunas_ddd.columns[0]
filtro_primeira_coluna = df_ddd_adicionado[primeira_coluna].notna()

# Retorna o dataframe completo filtrado por esse critério
df_filtrado = df_ddd_adicionado[filtro_primeira_coluna]

display(df_filtrado)


In [ ]:
# Um DataFrame df_ddd_adicionado com novos telefones corrigidos.

# Um subconjunto df_filtrado com escolas que têm pelo menos um telefone validado e reconstruído com DDD.

import pandas as pd
import numpy as np
import phonenumbers
import re


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)


df_high_mx = pd.read_csv('./csv/high_touch_mx.csv')
ladas_mexico = pd.read_csv('./csv/ladas_mexico.csv')


# Esta célula identifica todas as combinações de cidade + estado que possuem mais de uma clave LADA diferente.
# Primeiro, agrupa os dados por cidade e estado e conta quantas LADAs únicas existem em cada grupo.
# Em seguida, filtra apenas os pares com mais de uma LADA.
# Depois, faz um merge com outro agrupamento que coleta todas as LADAs únicas por par cidade + estado (em forma de lista),
# resultando em um DataFrame com as colunas: cidade, estado e a lista de LADAs correspondentes.
# Por fim, exibe quantas dessas combinações existem e mostra o DataFrame resultante.


claves_por_par = (
   ladas_mexico
   .groupby(['ciudad', 'estado'])['lada']
   .nunique()
   .reset_index()
   .query('lada > 1')
   .merge(
       ladas_mexico.groupby(['ciudad', 'estado'])['lada'].unique().reset_index().rename(columns={'lada': 'claves_lada'}),
       on=['ciudad', 'estado']
   )[['ciudad', 'estado', 'claves_lada']]
)


#print(f'Temos {len(claves_por_par)} combinações de cidade + estado com claves lada diferentes.')
#display(claves_por_par)
#display(ladas_mexico.head())


# Função de validação


def validar_telefone(numero: str, codigo_pais='MX'):
   if pd.isna(numero):
       return None
   numero = re.sub(r'\D', '', str(numero)).strip()
   try:
       parsed = phonenumbers.parse(numero, codigo_pais)
       if phonenumbers.is_valid_number(parsed):
           return numero
   except phonenumbers.NumberParseException:
       pass
   return None


# Normalização prévia dos DataFrames auxiliares
claves_por_par["estado"] = claves_por_par["estado"].astype(str).str.strip().str.lower()
claves_por_par["ciudad"] = claves_por_par["ciudad"].astype(str).str.strip().str.lower()


ladas_mexico["estado"] = ladas_mexico["estado"].astype(str).str.strip().str.lower()
ladas_mexico["ciudad"] = ladas_mexico["ciudad"].astype(str).str.strip().str.lower()


# Função aplicada por linha
def processar_linha(row, claves_por_par, ladas_mexico):
   telefone = row.get("telefone_scraping")
   estado = str(row.get("n_entidad", "")).strip().lower()
   cidade = str(row.get("city", "")).strip().lower()
   resultados = {}


   if isinstance(telefone, str) and len(telefone) in (7, 8):
       # Primeiro: tenta encontrar na claves_por_par
       mask_cp = (claves_por_par["estado"] == estado) & (claves_por_par["ciudad"] == cidade)
       rec_cp = claves_por_par[mask_cp]


       if not rec_cp.empty:
           ddds = rec_cp.iloc[0]["claves_lada"]
           for i, ddd in enumerate(ddds):
               numero = str(ddd) + telefone
               validado = validar_telefone(numero)
               if validado:
                   col = "tel_ddd_adicionado" if i == 0 else f"tel_ddd_adicionado_{i+1}"
                   resultados[col] = validado
           return pd.Series(resultados)


       # Caso contrário: tenta encontrar na ladas_mexico
       mask_lm = (ladas_mexico["estado"] == estado) & (ladas_mexico["ciudad"] == cidade)
       rec_lm = ladas_mexico[mask_lm]


       if not rec_lm.empty:
           ddd = rec_lm.iloc[0]["lada"]
           numero = str(ddd) + telefone
           validado = validar_telefone(numero)
           if validado:
               resultados["tel_ddd_adicionado"] = validado


   return pd.Series(resultados)


# Função principal
def adicionar_ddd(df_high_mx, claves_por_par, ladas_mexico):
   novos_ddds = df_high_mx.apply(
       processar_linha,
       axis=1,
       args=(claves_por_par, ladas_mexico)
   )
   df_ddd_adicionado = pd.concat([df_high_mx, novos_ddds], axis=1)
   return df_ddd_adicionado


# Execução
df_ddd_adicionado = adicionar_ddd(df_high_mx, claves_por_par, ladas_mexico)


# Filtra as colunas que contém 'tel_ddd_adicionado' no nome
colunas_ddd = df_ddd_adicionado.filter(like='tel_ddd_adicionado')


# Quantidade de colunas com 'tel_ddd_adicionado'
num_colunas = len(colunas_ddd.columns)


# Soma total de valores válidos (não nulos) em todas essas colunas
total_validos = colunas_ddd.notna().sum().sum()


print(f"Colunas 'tel_ddd_adicionado' criadas: {num_colunas}")
print(f"Total de valores válidos somados em todas as colunas: {total_validos}")


# Filtra linhas onde a primeira coluna 'tel_ddd_adicionado' tem valor válido (não nulo)
primeira_coluna = colunas_ddd.columns[0]
filtro_primeira_coluna = df_ddd_adicionado[primeira_coluna].notna()


# Retorna o dataframe completo filtrado por esse critério
df_filtrado = df_ddd_adicionado[filtro_primeira_coluna]


display(len(df_ddd_adicionado))

In [ ]:
print(df_ddd_adicionado['telefone_scraping'].notna().sum())
print(df_ddd_adicionado['tel_ddd_adicionado'].notna().sum())

In [ ]:
import phonenumbers
import re


# Cria uma cópia do DataFrame original para não modificar o original
df_final = df_ddd_adicionado.copy()


def limpar_e_validar(numero, codigo_pais='MX'):
   if pd.isna(numero):
       return None
   # Remove símbolos não numéricos e espaços
   numero = re.sub(r'\D', '', str(numero)).strip()
   try:
       parsed = phonenumbers.parse(numero, codigo_pais)
       if phonenumbers.is_valid_number(parsed):
           return numero
   except phonenumbers.NumberParseException:
       pass
   return None


# Aplica a validação e limpeza somente na coluna 'telefone_scraping'
df_final['telefone_scraping'] = df_final['telefone_scraping'].apply(limpar_e_validar)




num_validos_telefone = df_final['telefone_scraping'].notna().sum()
num_validos_ddd = df_final['tel_ddd_adicionado'].notna().sum()
soma = num_validos_telefone + num_validos_ddd


print(f"Números válidos em 'telefone_scraping': {num_validos_telefone}")
print(f"Valores não nulos em 'tel_ddd_adicionado': {num_validos_ddd}")
print(f"Foram adicionados {soma} números de telefone válidos.")


df_final.to_csv('df_final.csv', index=False)